In [66]:
# #DBSCAN
# """
# KMeans = simple, fast, needs k, works best for round clusters, bad with noise.
#
# DBSCAN = flexible shape, detects noise, no need for k, good for real-world messy data.
#
#
# ✅ Real-life example
# KMeans: Segment your coffee shop customers into 3 spending groups (low, medium, high spenders).
#
# DBSCAN: In a smart city, detect groups of pedestrians and cyclists moving together — they form clusters of random shapes, and stray objects (like a bird) are ignored as noise.
# """

In [67]:
#Concept of DBSCAN--Minpoints: points in circle of epsilon must be equal to minpoint or greater , epsilon:radius  , corepoint:all point which is taken randomly inside the circle have minpoint value, borderpoint: circle having only one data point

#COndition for making cluster-core point between points , and distance should be less the epsilon



DBSCAN MOVIE RECOMMENDATION SYSTEM

In [68]:
import pandas as pd
import numpy as np

import warnings #built in python module for the warnings generated by pandas ,numpy ,sklearn etc
warnings.filterwarnings('ignore') #used to filter the output clean


from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler

In [69]:
df_netflix=pd.read_csv('../../Assets/netflixTitles.csv')
df_amazon=pd.read_csv('../../Assets/amazontitle.csv')
df_hbo=pd.read_csv('../../Assets/hboTitle.csv')

In [70]:
df=pd.concat([df_netflix,df_amazon,df_hbo],axis=0)
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [71]:
df_movies=df.drop_duplicates()

In [72]:
df_movies.drop(['description', 'age_certification'], axis=1, inplace=True)


In [73]:
df['production_countries']


0             ['US']
1             ['US']
2             ['US']
3             ['GB']
4       ['GB', 'US']
            ...     
3289          ['PR']
3290          ['PA']
3291              []
3292              []
3293          ['US']
Name: production_countries, Length: 19015, dtype: object

In [74]:
df_movies['production_countries'] = df_movies['production_countries'].str.replace(r"\[", '', regex=True).str.replace(r"'", '', regex=True).str.replace(r"\]", '', regex=True)


In [75]:
df_movies['lead_prod_country'] = df_movies['production_countries'].str.split(',').str[0]


In [76]:
df_movies['prod_countries_cnt'] = df_movies['production_countries'].str.split(',').str.len()


In [77]:
df_movies['lead_prod_country'] = df_movies['lead_prod_country'].replace('', np.nan)


In [78]:
df_movies['lead_prod_country']


0        US
1        US
2        US
3        GB
4        GB
       ... 
3289     PR
3290     PA
3291    NaN
3292    NaN
3293     US
Name: lead_prod_country, Length: 18980, dtype: object

In [79]:
df_movies['genres']


0                                 ['documentation']
1                                ['drama', 'crime']
2       ['drama', 'action', 'thriller', 'european']
3                   ['fantasy', 'action', 'comedy']
4                                 ['war', 'action']
                           ...                     
3289                           ['romance', 'music']
3290                                     ['comedy']
3291                                     ['comedy']
3292                                     ['comedy']
3293                              ['documentation']
Name: genres, Length: 18980, dtype: object

In [80]:
df_movies['genres']=df_movies['genres'].str.replace(r"\[", '', regex=True).str.replace(r"'", '', regex=True).str.replace(r"\]", '', regex=True)

df_movies['main_genre'] = df_movies['genres'].str.split(',').str[0]
df_movies['main_genre'] = df_movies['main_genre'].replace('', np.nan)


In [81]:
df_movies['main_genre']


0       documentation
1               drama
2               drama
3             fantasy
4                 war
            ...      
3289          romance
3290           comedy
3291           comedy
3292           comedy
3293    documentation
Name: main_genre, Length: 18980, dtype: object

In [82]:
df_movies.drop(['genres', 'production_countries'], axis=1, inplace=True)


In [83]:
df_movies.shape


(18980, 14)

In [84]:
df_movies.isnull().sum()

id                        0
title                     1
type                      0
release_year              0
runtime                   0
seasons               14772
imdb_id                1394
imdb_score             1873
imdb_votes             1910
tmdb_popularity         670
tmdb_score             2656
lead_prod_country      1160
prod_countries_cnt        0
main_genre              321
dtype: int64

In [85]:
# Drop rows with any missing values to clean the dataset
df_movies.dropna(inplace=True)

# Set the 'title' column as the DataFrame index
df_movies.set_index('title', inplace=True)

# Drop the 'id' and 'imdb_id' columns as they are not needed for further analysis
df_movies.drop(['id', 'imdb_id'], axis=1, inplace=True)

In [86]:
# Create dummy variables for categorical columns ('type', 'lead_prod_country', 'main_genre')
dummies = pd.get_dummies(df_movies[['type', 'lead_prod_country', 'main_genre']], drop_first=True)

In [87]:
dummies.head()

,lead_prod_country_AR,lead_prod_country_AT,lead_prod_country_AU,lead_prod_country_BE,lead_prod_country_BR,lead_prod_country_CA,lead_prod_country_CH,lead_prod_country_CL,lead_prod_country_CN,lead_prod_country_CO,...,main_genre_history,main_genre_horror,main_genre_music,main_genre_reality,main_genre_romance,main_genre_scifi,main_genre_sport,main_genre_thriller,main_genre_war,main_genre_western
title,,,,,,,,,,,,,,,,,,,,,
Monty Python's Flying Circus,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Seinfeld,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Knight Rider,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
Thomas & Friends,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Saved by the Bell,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [88]:
# Concatenate the dummy variables with the original DataFrame
df_movies_dum = pd.concat([df_movies, dummies], axis=1)

# 14. Drop the original categorical columns after creating dummy variables
df_movies_dum.drop(['type', 'lead_prod_country', 'main_genre'], axis=1, inplace=True)


In [89]:
df_movies_dum.head()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,prod_countries_cnt,lead_prod_country_AR,lead_prod_country_AT,...,main_genre_history,main_genre_horror,main_genre_music,main_genre_reality,main_genre_romance,main_genre_scifi,main_genre_sport,main_genre_thriller,main_genre_war,main_genre_western
title,,,,,,,,,,,,,,,,,,,,,
Monty Python's Flying Circus,1969,30,4.0,8.8,73424.0,17.617,8.306,1,False,False,...,False,False,False,False,False,False,False,False,False,False
Seinfeld,1989,24,9.0,8.9,308824.0,130.213,8.301,1,False,False,...,False,False,False,False,False,False,False,False,False,False
Knight Rider,1982,51,4.0,6.9,34115.0,50.267,7.500,1,False,False,...,False,False,False,False,False,True,False,False,False,False
Thomas & Friends,1984,10,24.0,6.5,5104.0,42.196,6.500,1,False,False,...,False,False,False,False,False,False,False,False,False,False
Saved by the Bell,1989,23,5.0,7.1,35034.0,19.855,8.000,1,False,False,...,False,False,False,False,False,False,False,False,False,False


In [90]:
#SCALING
# Apply MinMaxScaler to scale the data for model training
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_movies_dum)
df_scaled = pd.DataFrame(df_scaled, columns=df_movies_dum.columns)

# Display the scaled DataFrame

df_scaled


,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,prod_countries_cnt,lead_prod_country_AR,lead_prod_country_AT,...,main_genre_history,main_genre_horror,main_genre_music,main_genre_reality,main_genre_romance,main_genre_scifi,main_genre_sport,main_genre_thriller,main_genre_war,main_genre_western
0,0.397727,0.168539,0.058824,0.9125,0.037009,0.007913,0.815870,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.625000,0.134831,0.156863,0.9250,0.155671,0.058490,0.815326,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.545455,0.286517,0.058824,0.6750,0.017194,0.022579,0.728261,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.568182,0.056180,0.450980,0.6250,0.002570,0.018954,0.619565,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.625000,0.129213,0.078431,0.7000,0.017658,0.008919,0.782609,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,0.988636,0.146067,0.000000,0.5000,0.000028,0.002064,0.456522,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3290,0.988636,0.258427,0.000000,0.5500,0.000027,0.002077,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3291,0.988636,0.185393,0.000000,0.5750,0.000017,0.000377,0.021739,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3292,0.988636,0.191011,0.019608,0.3125,0.000067,0.001158,0.510870,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
# Define the range of epsilon (eps) and minimum samples (min_samples) parameters for DBSCAN
eps_array = [0.2, 0.5, 1]  # List of different epsilon values (the maximum distance between two samples for one to be considered as in the neighborhood of the other)
min_samples_array = [5, 10, 30]  # List of different min_samples values (the number of samples in a neighborhood for a point to be considered as a core point)

# Iterate over each combination of eps and min_samples
for eps in eps_array:
    for min_samples in min_samples_array:
        # Initialize and fit the DBSCAN model with the current parameters
        clusterer = DBSCAN(eps=eps, min_samples=min_samples).fit(df_scaled)

        # Retrieve the cluster labels from the fitted model
        cluster_labels = clusterer.labels_

        # Check if the algorithm found only one cluster or marked all points as noise (-1 label for noise)
        if len(set(cluster_labels)) == 1:
            continue  # Skip this combination as it does not provide meaningful clusters

        # Calculate the silhouette score to evaluate the quality of the clustering
        silhouette_avg = silhouette_score(df_scaled, cluster_labels)

        # Print the current parameters, number of clusters, and the silhouette score
        print("For eps =", eps,
              "For min_samples =", min_samples,
              "Count clusters =", len(set(cluster_labels)),
              "The average silhouette_score is :", silhouette_avg)

For eps = 0.2 For min_samples = 5 Count clusters = 75 The average silhouette_score is : 0.4378840737098286
For eps = 0.2 For min_samples = 10 Count clusters = 37 The average silhouette_score is : 0.36601440046646755
For eps = 0.2 For min_samples = 30 Count clusters = 17 The average silhouette_score is : 0.23106054247198202
For eps = 0.5 For min_samples = 5 Count clusters = 91 The average silhouette_score is : 0.6019560501740351
For eps = 0.5 For min_samples = 10 Count clusters = 56 The average silhouette_score is : 0.5303679432698052
For eps = 0.5 For min_samples = 30 Count clusters = 21 The average silhouette_score is : 0.36228604161700484
For eps = 1 For min_samples = 5 Count clusters = 93 The average silhouette_score is : 0.6091664186394289
For eps = 1 For min_samples = 10 Count clusters = 57 The average silhouette_score is : 0.5362809971937993
For eps = 1 For min_samples = 30 Count clusters = 22 The average silhouette_score is : 0.37121300388037515


In [92]:
dbscan_model = DBSCAN(eps=1, min_samples=5).fit(df_scaled)
print("For eps =", 1,
      "For min_samples =", 5,
      "Count clusters =", len(set(dbscan_model.labels_)),
      "The average silhouette_score is :", silhouette_score(df_scaled, dbscan_model.labels_))

For eps = 1 For min_samples = 5 Count clusters = 93 The average silhouette_score is : 0.6091664186394289


In [93]:
df_movies['dbscan_clusters'] = dbscan_model.labels_


In [94]:
df_movies['dbscan_clusters'].value_counts()


dbscan_clusters
-1     387
 1     347
 13    267
 35    221
 5     145
      ... 
 86      5
 87      5
 69      5
 56      5
 78      5
Name: count, Length: 93, dtype: int64

MOVIE RECOMMENDATION FUNCTION

In [97]:
import random

def recommend_movie(movie_name: str):
    # Convert the input movie name to lowercase for case-insensitive matching
    movie_name = movie_name.lower()

    # Create a new column 'name' with lowercase movie names for comparison
    df_movies['name'] = df_movies.index.str.lower()

    # Find the movie that matches the input name
    movie = df_movies[df_movies['name'].str.contains(movie_name, na=False)]

    if not movie.empty:
        # Get the cluster label of the input movie
        cluster = movie['dbscan_clusters'].values[0]

        # Get all movies in the same cluster
        cluster_movies = df_movies[df_movies['dbscan_clusters'] == cluster]

        # If there are more than 5 movies in the cluster, randomly select 5
        if len(cluster_movies) >= 5:
            recommended_movies = random.sample(list(cluster_movies.index), 5)
        else:
            # If fewer than 5, return all the movies in the cluster
            recommended_movies = list(cluster_movies.index)

        # Print the recommended movies
        print('--- We can recommend you these movies ---')
        for m in recommended_movies:
            print(m)
    else:
        print('Movie not found in the database.')

In [102]:
s = input('Input movie name: ')

print("\n\n")
recommend_movie(s)




Movie not found in the database.
